In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [15]:
from tra.models import *
from tra.serializers import *
from tra.test_runs_processing import *
from rep_portal.api import *
from functools import reduce
from django.contrib.auth.models import User
from tra import utils
from stats.analyzer import Analyzer 
from tra.storage import *




In [14]:
def get_distinct_values_based_on_test_instance(test_instance: TestInstance):
        fields_dict = {}
        def get_distinct_values_and_serialize(field, model, serializer=None, order_by_param=None, key_override=None):
            order_by_param = order_by_param if order_by_param else field
            distinct_values = queryset.order_by(order_by_param).distinct(field).values_list(field, flat=True)
            objects = model.objects.filter(pk__in=distinct_values)
            data = serialize("json", objects)
            key = field if not key_override else key_override
            fields_dict[key] = json.loads(data)

        queryset = TestRun.objects.filter(test_instance=test_instance)
        fields_dict['analyzed'] = queryset.order_by('analyzed').distinct('analyzed').values_list("analyzed", flat=True)
        fields_dict['exec_trigger'] = queryset.order_by('exec_trigger').distinct('exec_trigger').values_list("exec_trigger", flat=True)
        fields_dict['exec_trigger'] = sorted(["null" if elem is None else elem for elem in fields_dict['exec_trigger']])
        get_distinct_values_and_serialize('fb', FeatureBuild, FeatureBuildSerializer, order_by_param='fb__name')
        get_distinct_values_and_serialize('result', TestRunResult, TestRunResultSerializer)
        get_distinct_values_and_serialize('testline_type', TestlineType, TestlineTypeSerializer)
        get_distinct_values_and_serialize('env_issue_type', EnvIssueType, EnvIssueTypeSerializer)
        get_distinct_values_and_serialize('analyzed_by', User, UserSerializer)
        return fields_dict
    
get_distinct_values_based_on_test_instance(TestInstance.objects.get(id=377))

{'analyzed': <QuerySet [False, True]>,
 'exec_trigger': ['CRT', 'null'],
 'fb': [{'model': 'tra.featurebuild',
   'pk': 'FB2307',
   'fields': {'start_time': '2023-03-28T22:00:00Z',
    'end_time': '2023-04-11T21:59:59Z'}},
  {'model': 'tra.featurebuild',
   'pk': 'FB2306',
   'fields': {'start_time': '2023-03-14T23:00:00Z',
    'end_time': '2023-03-28T21:59:59Z'}},
  {'model': 'tra.featurebuild',
   'pk': 'FB2305',
   'fields': {'start_time': '2023-02-28T23:00:00Z',
    'end_time': '2023-03-14T22:59:59Z'}},
  {'model': 'tra.featurebuild',
   'pk': 'FB2304',
   'fields': {'start_time': '2023-02-14T23:00:00Z',
    'end_time': '2023-02-28T22:59:59Z'}}],
 'result': [{'model': 'tra.testrunresult',
   'pk': 'environment issue',
   'fields': {}},
  {'model': 'tra.testrunresult', 'pk': 'passed', 'fields': {}}],
 'testline_type': [{'model': 'tra.testlinetype',
   'pk': 'CLOUD_5G_O_E_AP_LO_SANSA_FS_ECPRI_CMWV_TDD',
   'fields': {}}],
 'env_issue_type': [{'model': 'tra.envissuetype', 'pk': '', '

In [12]:
TestRun.objects.filter(log_file_url_ext__contains='15cedcaa-12f7-4ac0-8475-d2d77658c08a').exists()

True

In [5]:
def get_all_testruns_with_logs_available(test_instance):
    trs_with_logs = []
    for test_run in test_instance.test_runs.all().exclude(log_file_url='').exclude(log_file_url=None).filter(result=utils.get_passed_result_instance()).order_by('-end_time'):
        if test_run.has_ute_logs_available():
            trs_with_logs.append(test_run.id)
    return trs_with_logs

eligible_trs = []
for testset_filter in TestSetFilter.objects.all():
    if testset_filter.is_subscribed_by_anyone(): 
        test_instances = testset_filter.test_instances.all()
        for test_instance in test_instances:
            eligible_trs_per_ti = get_all_testruns_with_logs_available(test_instance)
            if eligible_trs_per_ti:
                eligible_trs.extend(eligible_trs_per_ti)

trs_queryset = TestRun.objects.filter(id__in=eligible_trs).order_by('ute_exec_url').values('ute_exec_url', 'execution_id').distinct()
html_storage = get_loghtml_storage_instance()
for testrun in trs_queryset:
    if not html_storage.exists(name=testrun['execution_id']):
        resp = html_storage.save(name=testrun['execution_id'], url=testrun['ute_exec_url'])
        break

--2023-04-03 16:01:09--  http://logs.ute.nsn-rdnet.net/cloud/execution/03d24b69-7838-455a-8d0b-5e42de1812ea/
Resolving lab-proxy.krk-lab.nsn-rdnet.net (lab-proxy.krk-lab.nsn-rdnet.net)... 10.158.100.2
Connecting to lab-proxy.krk-lab.nsn-rdnet.net (lab-proxy.krk-lab.nsn-rdnet.net)|10.158.100.2|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 540 [text/html]
Saving to: ‘index.html.tmp’

     0K                                                       100% 99.0M=0s

2023-04-03 16:01:09 (99.0 MB/s) - ‘index.html.tmp’ saved [540/540]

Loading robots.txt; please ignore errors.
--2023-04-03 16:01:09--  http://logs.ute.nsn-rdnet.net/robots.txt
Reusing existing connection to lab-proxy.krk-lab.nsn-rdnet.net:8080.
Proxy request sent, awaiting response... 404 Not Found
2023-04-03 16:01:10 ERROR 404: Not Found.

Removing index.html.tmp since it should be rejected.

--2023-04-03 16:01:10--  http://logs.ute.nsn-rdnet.net/cloud/execution/03d24b69-7838-455a-8d0b-5e42de1812ea/exe

     0K .......... .......... .......... .......... ..........  4%  405K 2s
    50K .......... .......... .......... .......... ..........  9%  305K 3s
   100K .......... .......... .......... .......... .......... 14%  285K 3s
   150K .......... .......... .......... .......... .......... 19%  363K 2s
   200K .......... .......... .......... .......... .......... 24%  336K 2s
   250K .......... .......... .......... .......... .......... 29%  368K 2s
   300K .......... .......... .......... .......... .......... 34%  462K 2s
   350K .......... .......... .......... .......... .......... 39%  631K 2s
   400K .......... .......... .......... .......... .......... 44%  633K 1s
   450K .......... .......... .......... .......... .......... 49%  644K 1s
   500K .......... .......... .......... .......... .......... 54%  878K 1s
   550K .......... .......... .......... .......... .......... 59%  515K 1s
   600K .......... .......... .......... .......... .......... 64%  621K 1s
   650K ....

Proxy request sent, awaiting response... 200 OK
Length: 1013 [text/html]
Saving to: ‘index.html.tmp’

     0K                                                       100%  225M=0s

2023-04-03 16:01:15 (225 MB/s) - ‘index.html.tmp’ saved [1013/1013]

Removing index.html.tmp since it should be rejected.

--2023-04-03 16:01:15--  http://logs.ute.nsn-rdnet.net/cloud/execution/03d24b69-7838-455a-8d0b-5e42de1812ea/execution_for_preparation_6f8c416c-f55c-4a0b-bd0d-33f9642ebeeb/test_results/admin_api_logs/RB-0005/
Reusing existing connection to lab-proxy.krk-lab.nsn-rdnet.net:8080.
Proxy request sent, awaiting response... 200 OK
Length: 1013 [text/html]
Saving to: ‘index.html.tmp’

     0K                                                       100%  186M=0s

2023-04-03 16:01:15 (186 MB/s) - ‘index.html.tmp’ saved [1013/1013]

Removing index.html.tmp since it should be rejected.

--2023-04-03 16:01:15--  http://logs.ute.nsn-rdnet.net/cloud/execution/03d24b69-7838-455a-8d0b-5e42de1812ea/execution_f

In [8]:
dirs, files = html_storage.listdir('')
dirs

['03d24b69-7838-455a-8d0b-5e42de1812ea']

In [17]:
# Entity.objects.order_by('foreign_key').values('foreign_key').distinct()
print(TestRun.objects.filter(id__in=eligible_trs).order_by('log_file_url').count())
print(TestRun.objects.filter(id__in=eligible_trs).order_by('log_file_url').values('log_file_url').distinct().count())
TestRun.objects.filter(id__in=eligible_trs).order_by('log_file_url').values('log_file_url', 'execution_id').distinct()

310
82


<QuerySet [{'log_file_url': 'http://logs.ute.nsn-rdnet.net/cloud/execution/03d24b69-7838-455a-8d0b-5e42de1812ea/execution_for_preparation_6f8c416c-f55c-4a0b-bd0d-33f9642ebeeb/test_results/log.html', 'execution_id': '03d24b69-7838-455a-8d0b-5e42de1812ea'}, {'log_file_url': 'http://logs.ute.nsn-rdnet.net/cloud/execution/04b31c3e-2205-4238-955e-1c55dd513ecf/execution_for_preparation_34b00c73-e62d-40e9-bd62-30eb1129bc5a/test_results/log.html', 'execution_id': '04b31c3e-2205-4238-955e-1c55dd513ecf'}, {'log_file_url': 'http://logs.ute.nsn-rdnet.net/cloud/execution/11bb321a-54e8-4430-9676-fb3718d6d185/execution_for_preparation_d1573cca-50ac-4942-8c45-681b6266e8ee/test_results/log.html', 'execution_id': '11bb321a-54e8-4430-9676-fb3718d6d185'}, {'log_file_url': 'http://logs.ute.nsn-rdnet.net/cloud/execution/15cedcaa-12f7-4ac0-8475-d2d77658c08a/execution_for_preparation_3c2ca44b-9e9a-465f-b3d0-67fb7fd51a7c/test_results/log.html', 'execution_id': '15cedcaa-12f7-4ac0-8475-d2d77658c08a'}, {'log_fil

In [18]:
user = User.objects.get(username='plinta')
fields_dict = {}
def get_distinct_values_and_serialize(field, model, serializer=None, order_by_param=None, key_override=None):
    order_by_param = order_by_param if order_by_param else field
    distinct_values = queryset.order_by(order_by_param).distinct(field).values_list(field, flat=True)
    objects = model.objects.filter(pk__in=distinct_values)
    print(objects)
    data = serialize("json", objects)
    key = field if not key_override else key_override
    fields_dict[key] = json.loads(data)

queryset = TestRun.objects.all()
tsfilters = TestSetFilter.objects.filter(subscribers=user)
queryset = queryset.filter(
    reduce(lambda q, tsfilter: q | Q(test_instance__test_set=tsfilter), tsfilters, Q())
)

fields_dict["tsfilters"] = json.loads(serialize("json", tsfilters))
fields_dict['analyzed'] = queryset.order_by('analyzed').distinct('analyzed').values_list("analyzed", flat=True)
get_distinct_values_and_serialize('test_instance', TestInstance, TestInstanceSerializer, order_by_param="test_instance_id")

<QuerySet [<TestInstance: 5GC001085_B_001_Feature_flag_activation_... on Trunk>, <TestInstance: 5GC001085_B_002_No_gnbIdLengthConfig_con... on Trunk>, <TestInstance: 5GC001085_B_003_Empty_gnbIdLengthConfig_... on Trunk>, <TestInstance: 5GC001085_B_005_Only_NCI_range_in_gnbIdL... on Trunk>, <TestInstance: 5GC001085_B_007_Only_PLMN_ID_in_gnbIdLen... on Trunk>, <TestInstance: 5GC001085_B_008_Both_NCI_Range_and_PLMN_... on Trunk>, <TestInstance: 5GC001085_B_009_Both_NCI_Range_and_PLMN_... on Trunk>, <TestInstance: 5GC001085_B_014_NRADJNRCELL_object_alrea... on Trunk>, <TestInstance: 5GC001085_B_015_NRADJNRCELL_object_alrea... on Trunk>, <TestInstance: 5GC001085_B_017_Unsupported_PLMN_receive... on Trunk>, <TestInstance: 5GC001085_B_018_Measurement_Report_with_... on Trunk>, <TestInstance: 5GC001085_B_023_Multiple_supported_PLMNs... on Trunk>, <TestInstance: 5GC001085_B_027_Check_potential_excessiv... on Trunk>, <TestInstance: CB008533_B_B1_001_Successfull_UE_call_bu... on Trunk>, <TestInst

In [9]:
testset_filter = TestSetFilter.objects.get(id=1)
filters = utils.get_filters_for_rp_api(testrun_result="not analyzed,environment issue", testset_filter=testset_filter)

auth_params = utils.get_rp_api_auth_params(testset_filter)
test_runs_data, _ = RepPortal(**auth_params).get_data_from_testruns(limit=5, filters=filters)

Starting new HTTPS connection (1): rep-portal.wroclaw.nsn-rdnet.net
Starting new HTTPS connection (1): rep-portal.ext.net.com


In [8]:
# def get_distinct_values_based_on_subscribed_regfilters(user: User):
#         fields_dict = {}
#         def get_distinct_values_and_serialize(field, model, serializer=None, order_by_param=None, key_override=None):
#             order_by_param = order_by_param if order_by_param else field
#             distinct_values = queryset.order_by(order_by_param).distinct(field).values_list(field, flat=True)
#             objects = model.objects.filter(pk__in=distinct_values)
#             data = serialize("json", objects)
#             key = field if not key_override else key_override
#             fields_dict[key] = json.loads(data)

#         queryset = TestRun.objects.all()
#         tsfilters = TestSetFilter.objects.filter(subscribers=user)
#         queryset = queryset.filter(
#             reduce(lambda q, tsfilter: q | Q(test_instance__test_set=tsfilter), tsfilters, Q())
#         )

#         fields_dict["tsfilters"] = json.loads(serialize("json", tsfilters))
#         fields_dict['analyzed'] = queryset.order_by('analyzed').distinct('analyzed').values_list("analyzed", flat=True)
#         get_distinct_values_and_serialize('test_instance', TestInstance, TestInstanceSerializer)
#         get_distinct_values_and_serialize('fb', FeatureBuild, FeatureBuildSerializer, order_by_param='fb__name')
#         get_distinct_values_and_serialize('result', TestRunResult, TestRunResultSerializer)
#         get_distinct_values_and_serialize('testline_type', TestlineType, TestlineTypeSerializer)
#         get_distinct_values_and_serialize('env_issue_type', EnvIssueType, EnvIssueTypeSerializer)
#         get_distinct_values_and_serialize('analyzed_by', User, UserSerializer)
#         get_distinct_values_and_serialize('test_instance__test_set__branch', Branch, key_override="branch",
#                                           order_by_param="test_instance__test_set__branch__name")
#         test_set_distinct_values = tsfilters.order_by('test_set_name').distinct('test_set_name').values_list('test_set_name', flat=True)
#         fields_dict['test_set_name'] = [{'pk': elem} for elem in list(test_set_distinct_values)]
#         return fields_dict
    
    
    
# get_distinct_values_based_on_subscribed_regfilters(User.objects.get(username="plinta"))

{'tsfilters': [{'model': 'tra.testsetfilter',
   'pk': 1,
   'fields': {'limit': 50,
    'test_set_name': '5GC001085-B_Intra-frequency_inter-gNB_neighbor_NRREL_addition_-_Previously_established_Xn',
    'test_lab_path': 'Root\\Test_Sets\\Trunk\\RAN_L2_SW_KRK_2\\5GC001085_ANR_for_SA_intra-NR_intra-frequency_UE_based',
    'branch': 'Trunk',
    'testline_types': ['CLOUD_5G_I_LO_AP_LO_SANSA_FS_ECPRI_CMWV_TDD'],
    'owners': [5, 1, 2],
    'subscribers': [5, 1, 2],
    'fail_message_type_groups': [2, 4, 6]}},
  {'model': 'tra.testsetfilter',
   'pk': 2,
   'fields': {'limit': 50,
    'test_set_name': 'CB008533_B_Multiple_UEs_Call_Setup_4CC_DL_CA',
    'test_lab_path': 'Root\\Test_Sets\\Trunk\\RAN_L2_SW_KRK_2\\CB008533_Extended_Inter_band_DL_FDD-TDD_FR1_CA_up_to_4CCs',
    'branch': 'Trunk',
    'testline_types': ['CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD'],
    'owners': [5, 2],
    'subscribers': [5, 1, 2],
    'fail_message_type_groups': [2, 4, 6]}},
  {'model': 'tra.testsetfilt

In [15]:
# for tsf in TestSetFilter.objects.all():
#     tsfs_similars = TestSetFilter.objects.exclude(id=tsf.id).filter(test_set_name=tsf.test_set_name,
#                                                     test_lab_path=tsf.test_lab_path)

#     if tsfs_similars.exists():
#         print("similar exists")
#         for tsfsimilar in tsfs_similars:
#             if tsfsimilar.test_instances.all().exists():
#                 print("test_instances_exists")
#             else:
#                 print("test_instances_do not exist")
#             for tsim_ti in tsfsimilar.test_instances.all():
#                 tsim_ti.test_set = tsf
#                 print(f"setting {tsf} as tsf for {tsim_ti}")

#                 tsim_ti.save()

#     tsf.save()

similar exists
test_instances_exists
setting Trunk - CLOUD_5G_I_LO_AP_LO_SANSA_FS_ECPRI_CMWV_TDD - CB008204-A as tsf for CB008204_A03_SA_PDCP_Classical_CA_Counte... on Trunk
similar exists
test_instances_exists
setting Trunk - CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD - CB008204-A as tsf for CB008204_A03_SA_PDCP_Classical_CA_Counte... on Trunk
setting Trunk - CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD - CB008204-A as tsf for CB008204_A09_SA_Multiple_bearer_scenario... on Trunk
setting Trunk - CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD - CB008204-A as tsf for CB008204_A13_NSA_ulDataPath_ulOverLte_Co... on Trunk
setting Trunk - CLOUD_5G_I_BOO_AP_KLLL_SANSA_L1B_ECPRI_CMWV_TDD - CB008204-A as tsf for CB008204_A10_NSA_ulDataPath_ulOverLteNr_... on Trunk


---


In [3]:
def parse_failmessagetypes(ids_list):
    querysets = []
    for _id in ids_list:
        fail_message_type_group = FailMessageTypeGroup.objects.get(id=_id)
        querysets.append(fail_message_type_group.fail_message_types.all())

    fail_message_types = list(chain(*querysets))
    fail_message_dict = {fm_type.regex: fm_type.name for fm_type in fail_message_types}
    return fail_message_dict

def prepare_filters_for_users_all_subs_regfilters(user):
    def _parse_branches(branches):
        branches_new = []q
        for branch in branches:
            if branch.lower() == "trunk":
                branches_new.append("SBTS00")
            else:
                branches_new.append(f"SBTS{branch}")
        return branches_new

    fields_dict = utils.get_distinct_values_based_on_subscribed_regfilters(user)
    test_sets =      [elem["pk"] for elem in fields_dict["test_set_name"]]
    testline_types = [elem["pk"] for elem in fields_dict["testline_type"]]
    branches =       [elem["pk"] for elem in fields_dict["branch"]]
    branches = _parse_branches(branches)
    results = ["environment issue", "not analyzed"]

    return {
        "test_set": ",".join(test_sets),
        "testline_type": ",".join(testline_types),
        "builds": ",".join(branches),
        "result": ",".join(results)
    }



In [33]:
filters = prepare_filters_for_users_all_subs_regfilters(User.objects.get(username="plinta"))
filters["limit"] = 5000
fail_message_dict = parse_failmessagetypes([1,2,3,4,5,6])

date_start, date_end = "2022-09-01", "2022-10-01" 
analyzer = Analyzer(fail_message_dict, filters)
dataframe = analyzer.get_data_from_rp()

# df, label = analyzer.get_data_filtered_by_date_ranges(date_start, date_end)
# data = analyzer.plot_runs_by_exception_types(df=df)

Starting new HTTPS connection (1): rep-portal.wroclaw.nsn-rdnet.net


In [34]:
dataframe_norm_failmsg = analyzer._get_normalized_exception_data(dataframe)

In [43]:
df_dated, _ = analyzer.get_data_filtered_by_date_ranges(dataframe_norm_failmsg, date_start, date_end)

In [44]:
from io import BytesIO
in_memory_excell = BytesIO()
df_dated.to_excel(in_memory_excell)
in_memory_excell.seek(0,0)
with open("dataframe.xlsx", 'wb') as f:
    f.write(in_memory_excell.read())

2